In [2]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-11.parquet
!wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv


--2026-01-19 16:18:08--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-11.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.165.71.145, 18.165.71.48, 18.165.71.56, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.165.71.145|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1164775 (1,1M) [binary/octet-stream]
Saving to: 'green_tripdata_2025-11.parquet.1'

     0K .......... .......... .......... .......... ..........  4%  245K 4s
    50K .......... .......... .......... .......... ..........  8%  286K 4s
   100K .......... .......... .......... .......... .......... 13%  616K 3s
   150K .......... .......... .......... .......... .......... 17%  846K 2s
   200K .......... .......... .......... .......... .......... 21%  662K 2s
   250K .......... .......... .......... .......... .......... 26%  639K 2s
   300K .......... .......... .......... .......... .......... 30% 1,3

In [3]:
import pandas as pd

In [4]:
df = pd.read_parquet("green_tripdata_2025-11.parquet")

In [5]:
zones = pd.read_csv("taxi_zone_lookup.csv")

In [6]:
df["lpep_pickup_datetime"] = pd.to_datetime(df["lpep_pickup_datetime"])

df_nov = df[
    (df["lpep_pickup_datetime"] >= "2025-11-01") &
    (df["lpep_pickup_datetime"] < "2025-12-01")
]

In [7]:
short_trips_count = df_nov[df_nov["trip_distance"] <= 1].shape[0]
short_trips_count

8007

In [8]:
df_valid = df_nov[df_nov["trip_distance"] < 100].copy()

df_valid["pickup_date"] = df_valid["lpep_pickup_datetime"].dt.date

daily_max = (
    df_valid
    .groupby("pickup_date")["trip_distance"]
    .max()
)

In [10]:
daily_max.sort_values(ascending=False).head(1)

pickup_date
2025-11-14    88.03
Name: trip_distance, dtype: float64

In [9]:
df_18 = df_nov[
    df_nov["lpep_pickup_datetime"].dt.date == pd.to_datetime("2025-11-18").date()
]

pickup_totals = (
    df_18
    .groupby("PULocationID")["total_amount"]
    .sum()
    .reset_index()
)

pickup_totals = pickup_totals.merge(
    zones,
    left_on="PULocationID",
    right_on="LocationID",
    how="left"
)

pickup_totals.sort_values("total_amount", ascending=False).head(1)[
    ["Zone", "total_amount"]
]

,Zone,total_amount
39,East Harlem North,9281.92


In [11]:
east_harlem_north_id = zones[zones["Zone"] == "East Harlem North"]["LocationID"].iloc[0]


In [12]:
df_ehn = df_nov[df_nov["PULocationID"] == east_harlem_north_id]

In [13]:
top_tip = (
    df_ehn
    .groupby("DOLocationID")["tip_amount"]
    .max()
    .reset_index()
)

top_tip = top_tip.merge(
    zones,
    left_on="DOLocationID",
    right_on="LocationID",
    how="left"
)

top_tip.sort_values("tip_amount", ascending=False).head(1)[
    ["Zone", "tip_amount"]
]


,Zone,tip_amount
134,Yorkville West,81.89
